# Project 1 - Medical Appointment No Shows

---

- Data: https://www.kaggle.com/datasets/joniarroba/noshowappointments?select=KaggleV2-May-2016.csv
- What if that possible to predict someone to no-show an appointment?
  
---
 

## Clean Data 

In [1]:
# Dependencies and Setup
import pandas as pd
from datetime import datetime

# Read in data
df = pd.read_csv('KaggleV2-May-2016.csv')

In [2]:
# Column names
df.columns


Index(['PatientId', 'AppointmentID', 'Gender', 'ScheduledDay',
       'AppointmentDay', 'Age', 'Neighbourhood', 'Scholarship', 'Hipertension',
       'Diabetes', 'Alcoholism', 'Handcap', 'SMS_received', 'No-show'],
      dtype='object')

In [3]:
# Rename columns for consistency and fix typo
df.rename(columns={'AppointmentID':'Appointment_ID',
                   'PatientId':'Patient_ID',
                   'ScheduledDay':'Scheduled_Day',
                   'AppointmentDay':'Appointment_Day',
                   'Hipertension':'Hypertension',
                   'Handcap':'Handicap',
                   'SMS_received':'SMS_Received',
                   'No-show':'No_Show'
                   },
                   inplace=True)
df.columns

Index(['Patient_ID', 'Appointment_ID', 'Gender', 'Scheduled_Day',
       'Appointment_Day', 'Age', 'Neighbourhood', 'Scholarship',
       'Hypertension', 'Diabetes', 'Alcoholism', 'Handicap', 'SMS_Received',
       'No_Show'],
      dtype='object')

In [4]:
# Show the first 5 rows of the dataset
df.head()

,Patient_ID,Appointment_ID,Gender,Scheduled_Day,Appointment_Day,Age,Neighbourhood,Scholarship,Hypertension,Diabetes,Alcoholism,Handicap,SMS_Received,No_Show
0,2.987250e+13,5642903,F,2016-04-29T18:38:08Z,2016-04-29T00:00:00Z,62,JARDIM DA PENHA,0,1,0,0,0,0,No
1,5.589978e+14,5642503,M,2016-04-29T16:08:27Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,0,0,0,0,0,No
2,4.262962e+12,5642549,F,2016-04-29T16:19:04Z,2016-04-29T00:00:00Z,62,MATA DA PRAIA,0,0,0,0,0,0,No
3,8.679512e+11,5642828,F,2016-04-29T17:29:31Z,2016-04-29T00:00:00Z,8,PONTAL DE CAMBURI,0,0,0,0,0,0,No
4,8.841186e+12,5642494,F,2016-04-29T16:07:23Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,1,1,0,0,0,No


In [5]:
# Show info about the dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110527 entries, 0 to 110526
Data columns (total 14 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   Patient_ID       110527 non-null  float64
 1   Appointment_ID   110527 non-null  int64  
 2   Gender           110527 non-null  object 
 3   Scheduled_Day    110527 non-null  object 
 4   Appointment_Day  110527 non-null  object 
 5   Age              110527 non-null  int64  
 6   Neighbourhood    110527 non-null  object 
 7   Scholarship      110527 non-null  int64  
 8   Hypertension     110527 non-null  int64  
 9   Diabetes         110527 non-null  int64  
 10  Alcoholism       110527 non-null  int64  
 11  Handicap         110527 non-null  int64  
 12  SMS_Received     110527 non-null  int64  
 13  No_Show          110527 non-null  object 
dtypes: float64(1), int64(8), object(5)
memory usage: 11.8+ MB


The data has no null values.

In [6]:
#check the duplicated in data
df.duplicated().sum()

0

The data has no duplicated values.

In [7]:
# Describe the data 
df.describe().T.style.background_gradient()

,count,mean,std,min,25%,50%,75%,max
Patient_ID,110527.000000,147496265710394.062500,256094920291738.843750,39217.844390,4172614444192.000000,31731838713978.000000,94391720898175.000000,999981631772427.000000
Appointment_ID,110527.000000,5675305.123427,71295.751540,5030230.000000,5640285.500000,5680573.000000,5725523.500000,5790484.000000
Age,110527.000000,37.088874,23.110205,-1.000000,18.000000,37.000000,55.000000,115.000000
Scholarship,110527.000000,0.098266,0.297675,0.000000,0.000000,0.000000,0.000000,1.000000
Hypertension,110527.000000,0.197246,0.397921,0.000000,0.000000,0.000000,0.000000,1.000000
Diabetes,110527.000000,0.071865,0.258265,0.000000,0.000000,0.000000,0.000000,1.000000
Alcoholism,110527.000000,0.030400,0.171686,0.000000,0.000000,0.000000,0.000000,1.000000
Handicap,110527.000000,0.022248,0.161543,0.000000,0.000000,0.000000,0.000000,4.000000
SMS_Received,110527.000000,0.321026,0.466873,0.000000,0.000000,0.000000,1.000000,1.000000


- Age : the average patient age is 37 years; the min value is -1 which may be an error
- Scholarship : 9.8 % of patients are enrolled in welfare program
- Hypertension : 19.7 % of patents have hypertension
- Diabetes : 7.2% of patents have diabetes
- Alcoholism : 3.0% of patents have alcoholism
- Handcap : 2.2% of patents have a Handicap
- SMS_received : 32.1% of the patients received one or more SMS messages.

In [8]:
# Drop the row when the age value is -1
df=df[df['Age']!=-1]
df = df.reset_index(drop=True)

In [9]:
# Describe each 'object' type 
df.describe(include='object').T

,count,unique,top,freq
Gender,110526,2,F,71839
Scheduled_Day,110526,103548,2016-05-06T07:09:54Z,24
Appointment_Day,110526,27,2016-06-06T00:00:00Z,4691
Neighbourhood,110526,81,JARDIM CAMBURI,7717
No_Show,110526,2,No,88207


- Gender : There are 2 unique values in this column. The most frequent gender is Female (F), appearing 71,839 times.
- Neighbourhood : There are 81 unique values in this column. The most frequent neighbourhood is JARDIM CAMBURI, appearing 7,717 times.
- No_Show : There are 2 unique values in this column. The most frequent value is No, appearing 88,207 times. This means that most appointments were kept.

## Add columns
Add columns to be used for analysis.
- No_Show_Boolean : Use True and False instead of Yes and No
- Sevearl time columns : Break Scheduled Day and Appointment Day into Dates, Times, Day of Week, Month of Year
- Days_Between : Find days between Scheduled Day and Appointment Day
- Lat and long coordinates for neighborhoods, using Geoapify API (Bonus points!!)

### Boolean option for No_Show

In [10]:
# Add a Boolean option for No_Show
pd.set_option('future.no_silent_downcasting', True)
df['No_Show_Boolean'] = df['No_Show'].replace({'No': False, 'Yes': True})
df.head()

,Patient_ID,Appointment_ID,Gender,Scheduled_Day,Appointment_Day,Age,Neighbourhood,Scholarship,Hypertension,Diabetes,Alcoholism,Handicap,SMS_Received,No_Show,No_Show_Boolean
0,2.987250e+13,5642903,F,2016-04-29T18:38:08Z,2016-04-29T00:00:00Z,62,JARDIM DA PENHA,0,1,0,0,0,0,No,False
1,5.589978e+14,5642503,M,2016-04-29T16:08:27Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,0,0,0,0,0,No,False
2,4.262962e+12,5642549,F,2016-04-29T16:19:04Z,2016-04-29T00:00:00Z,62,MATA DA PRAIA,0,0,0,0,0,0,No,False
3,8.679512e+11,5642828,F,2016-04-29T17:29:31Z,2016-04-29T00:00:00Z,8,PONTAL DE CAMBURI,0,0,0,0,0,0,No,False
4,8.841186e+12,5642494,F,2016-04-29T16:07:23Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,1,1,0,0,0,No,False


### Add various date and time columns for Scheduled_Day and Appointment_Day

In [11]:
# Convert the Scheduled_Day and Appointment_Day to datetime
df['Scheduled_Day'] = pd.to_datetime(df['Scheduled_Day'])
df['Appointment_Day'] = pd.to_datetime(df['Appointment_Day'])

# Split Scheduled_Day column into date, hour, day of week, month
df['Scheduled_Day_Date'] = df['Scheduled_Day'].dt.strftime('%Y-%m-%d')
df['Scheduled_Hour'] = df['Scheduled_Day'].dt.hour
df['Scheduled_Day_of_Week'] = df['Scheduled_Day'].dt.day_name()
df['Scheduled_Month'] = df['Scheduled_Day'].dt.month

# Remove timestamp from Appointment_Day (since it is set at midnight for all entries)
df['Appointment_Day_Date'] = df['Appointment_Day'].dt.strftime('%Y-%m-%d')

# Split Appointment_Day column into day of week and month
df['Appointment_Day_of_Week'] = df['Appointment_Day'].dt.day_name()
df['Appointment_Month'] = df['Appointment_Day'].dt.month

df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110526 entries, 0 to 110525
Data columns (total 22 columns):
 #   Column                   Non-Null Count   Dtype              
---  ------                   --------------   -----              
 0   Patient_ID               110526 non-null  float64            
 1   Appointment_ID           110526 non-null  int64              
 2   Gender                   110526 non-null  object             
 3   Scheduled_Day            110526 non-null  datetime64[ns, UTC]
 4   Appointment_Day          110526 non-null  datetime64[ns, UTC]
 5   Age                      110526 non-null  int64              
 6   Neighbourhood            110526 non-null  object             
 7   Scholarship              110526 non-null  int64              
 8   Hypertension             110526 non-null  int64              
 9   Diabetes                 110526 non-null  int64              
 10  Alcoholism               110526 non-null  int64              
 11  Handicap     

,Patient_ID,Appointment_ID,Gender,Scheduled_Day,Appointment_Day,Age,Neighbourhood,Scholarship,Hypertension,Diabetes,...,SMS_Received,No_Show,No_Show_Boolean,Scheduled_Day_Date,Scheduled_Hour,Scheduled_Day_of_Week,Scheduled_Month,Appointment_Day_Date,Appointment_Day_of_Week,Appointment_Month
0,2.987250e+13,5642903,F,2016-04-29 18:38:08+00:00,2016-04-29 00:00:00+00:00,62,JARDIM DA PENHA,0,1,0,...,0,No,False,2016-04-29,18,Friday,4,2016-04-29,Friday,4
1,5.589978e+14,5642503,M,2016-04-29 16:08:27+00:00,2016-04-29 00:00:00+00:00,56,JARDIM DA PENHA,0,0,0,...,0,No,False,2016-04-29,16,Friday,4,2016-04-29,Friday,4
2,4.262962e+12,5642549,F,2016-04-29 16:19:04+00:00,2016-04-29 00:00:00+00:00,62,MATA DA PRAIA,0,0,0,...,0,No,False,2016-04-29,16,Friday,4,2016-04-29,Friday,4
3,8.679512e+11,5642828,F,2016-04-29 17:29:31+00:00,2016-04-29 00:00:00+00:00,8,PONTAL DE CAMBURI,0,0,0,...,0,No,False,2016-04-29,17,Friday,4,2016-04-29,Friday,4
4,8.841186e+12,5642494,F,2016-04-29 16:07:23+00:00,2016-04-29 00:00:00+00:00,56,JARDIM DA PENHA,0,1,1,...,0,No,False,2016-04-29,16,Friday,4,2016-04-29,Friday,4


In [12]:
# Add column for days between scheduled day and appointment day
df['Days_Between'] = ''

In [13]:
# Create functon to find days between two dates
def days_between(d1, d2):
    d1 = datetime.strptime(d1, '%Y-%m-%d')
    d2 = datetime.strptime(d2, '%Y-%m-%d')
    return abs((d2 - d1).days)

# Use functon to fill in Days_Between column
for i in range(df.shape[0]):
    df.loc[i,'Days_Between'] = days_between(df['Scheduled_Day_Date'][i],df['Appointment_Day_Date'][i])

# Check min and max days between
print("Min days between: " + str(df['Days_Between'].min()))
print("Max days between: " + str(df['Days_Between'].max()))

Min days between: 0
Max days between: 179


### Use Geoapify API to find lat and long of neighborhoods (Bonus!!)

In [20]:
# Dependencies
import requests
import json

# Import the API key
from api_key import geoapify_key

# List of neigborhoods
neighborhoods = df['Neighbourhood'].unique().tolist()
lat_list = []
lon_list = []

# base URL
base_url = "https://api.geoapify.com/v1/geocode/search"

# params
params = {
        "apiKey": geoapify_key
    }

# use Geoapify API to find lat and lon for each neighborhood
for neighborhood in neighborhoods:
    # Set the search parameters
    target_city = neighborhood + ", Brazil"
    params["text"] = target_city

    # Run request
    response = requests.get(base_url, params=params).json()

    # Print the json (pretty printed)
    #print(json.dumps(response, indent=4, sort_keys=True))

    # Extract lat/lon
    lat = response["features"][0]["properties"]["lat"]
    lon = response["features"][0]["properties"]["lon"]

    # Print results
    # print(f"{target_city}: {lat}, {lon}")

    lat_list.append(lat)
    lon_list.append(lon)

neighborhood_locations_df = pd.DataFrame({'Neighbourhood': neighborhoods, 'lat': lat_list, 'lon': lon_list})
neighborhood_locations_df


,Neighbourhood,lat,lon
0,JARDIM DA PENHA,-23.514826,-46.552112
1,MATA DA PRAIA,-22.788662,-43.079799
2,PONTAL DE CAMBURI,-23.354233,-44.725856
3,REPÚBLICA,-20.269941,-40.293034
4,GOIABEIRAS,-20.270612,-40.302859
...,...,...,...
76,FRADINHOS,-20.307131,-40.326980
77,NAZARETH,-20.310290,-40.316110
78,AEROPORTO,-3.775718,-38.527795
79,ILHAS OCEÂNICAS DE TRINDADE,39.523813,-87.128568


In [25]:
neighborhood_locations_df[neighborhood_locations_df["Neighbourhood"]== "ILHAS OCEÂNICAS DE TRINDADE"]

,Neighbourhood,lat,lon
79,ILHAS OCEÂNICAS DE TRINDADE,39.523813,-87.128568


In [26]:
neighborhood_locations_df[neighborhood_locations_df["Neighbourhood"]== ""]

,Neighbourhood,lat,lon
72,DO CABRAL,39.523813,-87.128568


In [30]:
# correct lat and lon manually

# ILHAS OCEÂNICAS DE TRINDADE
neighborhood_locations_df.loc[79,"lat"] = -23.350815
neighborhood_locations_df.loc[79,"lon"] = -44.724037 

# DO CABRAL
neighborhood_locations_df.loc[72,"lat"] = -20.3155
neighborhood_locations_df.loc[72,"lon"] = -40.3478 

In [31]:
df_merged = pd.merge(df, neighborhood_locations_df, how = 'left', on = 'Neighbourhood')
df_merged

,Patient_ID,Appointment_ID,Gender,Scheduled_Day,Appointment_Day,Age,Neighbourhood,Scholarship,Hypertension,Diabetes,...,Scheduled_Day_of_Week,Scheduled_Month,Appointment_Day_Date,Appointment_Day_of_Week,Appointment_Month,Days_Between,lat_x,lon_x,lat_y,lon_y
0,2.987250e+13,5642903,F,2016-04-29 18:38:08+00:00,2016-04-29 00:00:00+00:00,62,JARDIM DA PENHA,0,1,0,...,Friday,4,2016-04-29,Friday,4,0,-23.514826,-46.552112,-23.514826,-46.552112
1,5.589978e+14,5642503,M,2016-04-29 16:08:27+00:00,2016-04-29 00:00:00+00:00,56,JARDIM DA PENHA,0,0,0,...,Friday,4,2016-04-29,Friday,4,0,-23.514826,-46.552112,-23.514826,-46.552112
2,4.262962e+12,5642549,F,2016-04-29 16:19:04+00:00,2016-04-29 00:00:00+00:00,62,MATA DA PRAIA,0,0,0,...,Friday,4,2016-04-29,Friday,4,0,-22.788662,-43.079799,-22.788662,-43.079799
3,8.679512e+11,5642828,F,2016-04-29 17:29:31+00:00,2016-04-29 00:00:00+00:00,8,PONTAL DE CAMBURI,0,0,0,...,Friday,4,2016-04-29,Friday,4,0,-23.354233,-44.725856,-23.354233,-44.725856
4,8.841186e+12,5642494,F,2016-04-29 16:07:23+00:00,2016-04-29 00:00:00+00:00,56,JARDIM DA PENHA,0,1,1,...,Friday,4,2016-04-29,Friday,4,0,-23.514826,-46.552112,-23.514826,-46.552112
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110521,2.572134e+12,5651768,F,2016-05-03 09:15:35+00:00,2016-06-07 00:00:00+00:00,56,MARIA ORTIZ,0,0,0,...,Tuesday,5,2016-06-07,Tuesday,6,35,-20.257506,-40.302348,-20.257506,-40.302348
110522,3.596266e+12,5650093,F,2016-05-03 07:27:33+00:00,2016-06-07 00:00:00+00:00,51,MARIA ORTIZ,0,0,0,...,Tuesday,5,2016-06-07,Tuesday,6,35,-20.257506,-40.302348,-20.257506,-40.302348
110523,1.557663e+13,5630692,F,2016-04-27 16:03:52+00:00,2016-06-07 00:00:00+00:00,21,MARIA ORTIZ,0,0,0,...,Wednesday,4,2016-06-07,Tuesday,6,41,-20.257506,-40.302348,-20.257506,-40.302348
110524,9.213493e+13,5630323,F,2016-04-27 15:09:23+00:00,2016-06-07 00:00:00+00:00,38,MARIA ORTIZ,0,0,0,...,Wednesday,4,2016-06-07,Tuesday,6,41,-20.257506,-40.302348,-20.257506,-40.302348


### Save cleaned dataset with added columns

In [32]:
# Save the cleaned data for future use
df_merged.to_csv('data_cleaned.csv', index=False)  

## Load Cleaned Data for Analysis....(do this in separate notebook)

In [17]:
# Dependencies and Setup
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
from scipy.stats import linregress
import numpy as np

# Read in data
df = pd.read_csv('data_cleaned.csv')
df.head()

,Patient_ID,Appointment_ID,Gender,Scheduled_Day,Appointment_Day,Age,Neighbourhood,Scholarship,Hypertension,Diabetes,...,Scheduled_Day_Date,Scheduled_Hour,Scheduled_Day_of_Week,Scheduled_Month,Appointment_Day_Date,Appointment_Day_of_Week,Appointment_Month,Days_Between,lat,lon
0,2.987250e+13,5642903,F,2016-04-29 18:38:08+00:00,2016-04-29 00:00:00+00:00,62,JARDIM DA PENHA,0,1,0,...,2016-04-29,18,Friday,4,2016-04-29,Friday,4,0,-23.514826,-46.552112
1,5.589978e+14,5642503,M,2016-04-29 16:08:27+00:00,2016-04-29 00:00:00+00:00,56,JARDIM DA PENHA,0,0,0,...,2016-04-29,16,Friday,4,2016-04-29,Friday,4,0,-23.514826,-46.552112
2,4.262962e+12,5642549,F,2016-04-29 16:19:04+00:00,2016-04-29 00:00:00+00:00,62,MATA DA PRAIA,0,0,0,...,2016-04-29,16,Friday,4,2016-04-29,Friday,4,0,-22.788662,-43.079799
3,8.679512e+11,5642828,F,2016-04-29 17:29:31+00:00,2016-04-29 00:00:00+00:00,8,PONTAL DE CAMBURI,0,0,0,...,2016-04-29,17,Friday,4,2016-04-29,Friday,4,0,-23.354233,-44.725856
4,8.841186e+12,5642494,F,2016-04-29 16:07:23+00:00,2016-04-29 00:00:00+00:00,56,JARDIM DA PENHA,0,1,1,...,2016-04-29,16,Friday,4,2016-04-29,Friday,4,0,-23.514826,-46.552112


In [18]:
# Show info about the dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110526 entries, 0 to 110525
Data columns (total 25 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   Patient_ID               110526 non-null  float64
 1   Appointment_ID           110526 non-null  int64  
 2   Gender                   110526 non-null  object 
 3   Scheduled_Day            110526 non-null  object 
 4   Appointment_Day          110526 non-null  object 
 5   Age                      110526 non-null  int64  
 6   Neighbourhood            110526 non-null  object 
 7   Scholarship              110526 non-null  int64  
 8   Hypertension             110526 non-null  int64  
 9   Diabetes                 110526 non-null  int64  
 10  Alcoholism               110526 non-null  int64  
 11  Handicap                 110526 non-null  int64  
 12  SMS_Received             110526 non-null  int64  
 13  No_Show                  110526 non-null  object 
 14  No_S